In [1]:
import os
import sys
import yaml
import json
import pickle
import dagshub
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.metrics import  accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve


In [2]:
tr_df = pd.read_csv('../../../data/feature/ks_train.csv')
val_df = pd.read_csv('../../../data/feature/ks_validation.csv')

In [3]:
with open('parameters.json', 'r')as pf:
    params = json.load(pf)

In [4]:
xgbc = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='multi:softmax', num_class=3,
                    silent=True, nthread=1, **params)

In [7]:
xgbc.fit(tr_df.iloc[:, :-1], tr_df.iloc[:, -1])

[18:08:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[18:08:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1.0, gamma=5, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.02, max_delta_step=0, max_depth=3,
              min_child_weight=5, missing=nan, monotone_constraints='()',
              n_estimators=600, n_jobs=1, nthread=1, num_class=3,
              num_parallel_tree=1, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=None, silent=True,
              subsample=1.0, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [9]:
with open('xgb-classifier.pkl','wb') as pf:
    pickle.dump(xgbc, pf)

In [16]:
val_len = val_df.shape[0]
tr_sub = tr_df.sample(n=val_len, random_state=443)
y_tr_pred = xgbc.predict(tr_sub.iloc[:, :-1])
y_train = tr_sub.iloc[:, -1]

y_ts_pred = xgbc.predict(val_df.iloc[:, :-1])
y_test = val_df.iloc[:, -1]

In [20]:
precision_score(y_train, y_tr_pred, average='weighted')

0.584676910853922

In [23]:
with open('score.json','w') as pf:
    json.dump({'train' : {'accuracy' : accuracy_score(y_train, y_tr_pred), 'precision' : precision_score(y_train, y_tr_pred, average='weighted'), 'recall' : recall_score(y_train, y_tr_pred, average='weighted'), 'f1-score' : f1_score(y_train, y_tr_pred, average='weighted')},
    'test' : {'accuracy' : accuracy_score(y_test, y_ts_pred), 'precision' : precision_score(y_test, y_ts_pred, average='weighted'), 'recall' : recall_score(y_test, y_ts_pred, average='weighted'), 'f1-score' : f1_score(y_test, y_ts_pred, average='weighted')}
    }, pf)